### Importing packages and modules

In [1]:
# module for building the pyomo model
import pyomo.environ as pe
# module for solving the pyomo model
import pyomo.opt as po

### Create the model

In [2]:
model = pe.ConcreteModel()

Order to build the model:
1. Sets
1. Parameters
1. Variables
1. Objective function
1. Constraints

#### Sets




In [3]:
N_ROW = 8
N_COL = 8
MAX_STEPS = 2 * N_ROW * N_COL
N_TRUCKS = 4

In [4]:
model.row = pe.Set(initialize=[i for i in range(N_ROW)])
model.column = pe.Set(initialize=[j for j in range(N_COL)])
model.truck = pe.Set(initialize=[k for k in range(N_TRUCKS)])
model.step = pe.Set(initialize=[i for i in range(MAX_STEPS)])

#### Parameters

In [5]:
empty_grid = {}

# Iterate over the rows and columns of the 3x3grid
for i in range(N_ROW):
    for j in range(N_COL):
        empty_grid[(i, j)] = 0

$CP(i,j)$: Cantidad de basura que hay en la parada regular en (i,j)

In [6]:
dirt_regular_stops = empty_grid

stop_position_1 = (1,2)
dirt_regular_stops[stop_position_1] = 3
stop_position_2 = (4,2)
dirt_regular_stops[stop_position_2] = 5
stop_position_3 = (0,4)
dirt_regular_stops[stop_position_3] = 2
stop_position_4 = (2, 3)
dirt_regular_stops[stop_position_4] = 4
stop_position_5 = (3, 1)
dirt_regular_stops[stop_position_5] = 3
stop_position_6 = (4, 4)
dirt_regular_stops[stop_position_6] = 4
stop_position_7 = (1, 6)
dirt_regular_stops[stop_position_7] = 2
stop_position_8 = (5, 2)
dirt_regular_stops[stop_position_8] = 4
stop_position_9 = (6, 5)
dirt_regular_stops[stop_position_9] = 3
stop_position_10 = (7, 1)
dirt_regular_stops[stop_position_10] = 5
stop_position_11 = (3, 7)
dirt_regular_stops[stop_position_11] = 1
stop_position_12 = (6, 0)
dirt_regular_stops[stop_position_12] = 2
stop_position_13 = (2, 5)
dirt_regular_stops[stop_position_13] = 3
# stop_position_14 = (0, 7)
# dirt_regular_stops[stop_position_14] = 2

model.dirt_regular_stops = pe.Param(model.row, model.column, initialize = dirt_regular_stops)

$D(i,j)$: Depósito en (i,j)

In [7]:
deposit = empty_grid
deposit_position = (0, 0)
deposit[deposit_position] = 1

model.deposit = pe.Param(model.row, model.column, initialize=deposit)

$C_k$: capacidad del vehículo k

In [8]:
capacity = [9+k for k in model.truck]

model.capacity = pe.Param(model.truck, initialize = capacity)

#### Variables



In [9]:
# Truck route, ones and zeros to see where it passes
model.r = pe.Var(model.row, model.column, model.truck, model.step, within=pe.Binary)

# Amount of garbage in the truck
model.b = pe.Var(model.truck, model.step, within=pe.NonNegativeReals)

# Binary variable indicating if the truck can collect more garbage
model.can_collect = pe.Var(model.truck, within=pe.Binary)

# Binary variable indicating if all the garbage has been collected
model.all_dirt_collected = pe.Var(model.step, within=pe.Binary)

# Number of steps taken to return to the depot
model.return_step = pe.Var(model.truck, within=pe.NonNegativeIntegers, bounds=(0, MAX_STEPS))

# Mark if the garbage from a cell has been collected at each step
model.is_collected = pe.Var(model.row, model.column, model.step, domain=pe.Binary)


#### Objective Function


In [10]:
# Objective function: Minimize the return steps, considering returning to the depot as soon as possible,
# collecting all the garbage, since the depot can only be visited twice and the second visit can only happen
# once all the garbage has been collected

model.obj = pe.Objective(
    expr=sum(model.return_step[k] for k in model.truck), 
    sense=pe.minimize
)

#### Constraints

In [11]:
# Remove previous constraints
model.del_component('constraints')
model.constraints = pe.ConstraintList()

# 1. The truck starts at the depot
for k in model.truck:
    model.constraints.add(model.r[deposit_position[0], deposit_position[1], k, 0] == 1)

# 2. Manhattan movement constraint (moves only to adjacent cells)
for k in model.truck:
    for s in model.step:
        for i in model.row:
            for j in model.column:
                if s > 0:
                    model.constraints.add(
                        model.r[i, j, k, s] <= 
                        (model.r[i-1, j, k, s-1] if i > 0 else 0) +  # Up
                        (model.r[i+1, j, k, s-1] if i < N_ROW - 1 else 0) +  # Down
                        (model.r[i, j-1, k, s-1] if j > 0 else 0) +  # Left
                        (model.r[i, j+1, k, s-1] if j < N_COL - 1 else 0)  # Right
                    )

# 3. Constraint of only one cell occupied at each time step
for k in model.truck:
    for s in model.step:
        model.constraints.add(
            sum(model.r[i, j, k, s] for i in model.row for j in model.column) <= 1
        )

# 4. Truck capacity constraint
for k in model.truck:
    # for s in model.step:
    model.constraints.add(
        sum(model.dirt_regular_stops[i, j] * model.r[i, j, k, s] for i in model.row for j in model.column for s in model.step) <= model.capacity[k]
    )

# 5. Constraint to update the truck load when collecting garbage
# Restrict that the load only accumulates the first time a garbage cell is visited
for k in model.truck:
    for s in model.step:
        if s > 0:
            model.constraints.add(
                model.b[k, s] == model.b[k, s-1] + sum(
                    model.dirt_regular_stops[i, j] * model.r[i, j, k, s] * (1 - model.is_collected[i, j, s-1])
                    for i in model.row for j in model.column
                )
            )
        else:
            model.constraints.add(model.b[k, s] == 0)  # Initially empty

# 6. Constraint to ensure that a garbage cell is marked as collected only once 
BIG_M = 10000
for s in model.step:
    for i in model.row:
        for j in model.column:
            for k in model.truck:
                if s > 0:
                    model.constraints.add(
                        model.is_collected[i, j, s] - model.can_collect[k] >= model.r[i, j, k, s]
                    )
                    model.constraints.add(
                        model.capacity[k] - (model.b[k, s] + model.dirt_regular_stops[i, j]) >= BIG_M * (model.can_collect[k] - 1)
                    )
                    # Once activated, it remains in subsequent steps
                    model.constraints.add(
                        model.is_collected[i, j, s] >= model.is_collected[i, j, s-1]
                    )

# 7. Activate 'all_dirt_collected' when all garbage has been collected
small_m = 1*10**-4

# Can be 0 or 1 if it is 0
# It is 0 if it is != 0
for s in model.step:
    model.constraints.add(
        BIG_M * (1 - model.all_dirt_collected[s]) >= sum(model.dirt_regular_stops[i, j] for i in model.row for j in model.column) - sum(model.b[k, s] for k in model.truck)
    )

# It is 1 if it is 0
# Can be 0 or 1 if it is != 0
for s in model.step:
    model.constraints.add(
        small_m * (1-model.all_dirt_collected[s]) <= sum(model.dirt_regular_stops[i, j] for i in model.row for j in model.column) - sum(model.b[k, s] for k in model.truck)
    )

# 8. The depot can only be visited twice by eack truck
for k in model.truck:
    model.constraints.add(
        sum(model.r[deposit_position[0], deposit_position[1], k, s] for s in model.step) == 2
    )

# 9. Constraint to ensure that the second visit to the depot is only after all garbage has been collected for each truck
for k in model.truck:
    for s in model.step:
        if s > 0:
            model.constraints.add(
                model.r[deposit_position[0], deposit_position[1], k, s] <= model.all_dirt_collected[s]
            )
        
# 10. Minimization of the number of steps back to the depot for each truck
for k in model.truck:
    model.constraints.add(
        model.return_step[k] == sum(s * model.r[deposit_position[0], deposit_position[1], k, s] for s in model.step)
    )

# 11. Stop condition. For each step after having returned to the deposit, the truck will no longer be on the map
for k in model.truck:
    for s in model.step:
        if s > 1:
            model.constraints.add(
                sum(model.r[i, j, k, s] for i in model.row for j in model.column) <= (1 - model.r[deposit_position[0], deposit_position[1], k, s-1])  # Fuerza r[i, j, s] = 0 si hemos vuelto al depósito
            )

# 12. At each cell, garbage is collected only once
for i in model.row:
    for j in model.column:
        model.constraints.add(
            sum(model.r[i, j, k, s] * model.dirt_regular_stops[i, j] for k in model.truck for s in model.step) <= model.dirt_regular_stops[i, j]
        )


In [12]:
solver = po.SolverFactory('gurobi')
results = solver.solve (model, tee=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-14
Read LP format model from file C:\Users\sofin\AppData\Local\Temp\tmpypnjipqz.pyomo.lp
Reading time = 0.88 seconds
x1: 131912 rows, 41609 columns, 456056 nonzeros
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 131912 rows, 41609 columns and 456056 nonzeros
Model fingerprint: 0xa718a747
Model has 508 quadratic constraints
Variable types: 513 continuous, 41096 integer (41092 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+04]
  QMatrix range    [1e+00, 5e+00]
  QLMatrix range   [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+04]
Presolve removed 113923 rows and 24782 columns
Presolve time: 1.65s
Presolved:

In [13]:
# Extract the route from model.r
route = []

# Iterate over each truck, each cell, and each step (if applicable)
# for s in range(int(model.return_step.value) + 1):
for k in model.truck:
    for s in range(MAX_STEPS):
        for i in model.row:
            for j in model.column:
                if pe.value(model.r[i, j, k, s]) > 0.5:  # Check if the variable is 1
                    route.append((i, j, k, s))

# Display the route
counter = 0
for k in model.truck:
    print(f"\nTruck {k} with capacity {model.capacity[k]}")
    for point in route:
        if k == point[2]:
            print(f"Step {point[3]}: Truck {point[2]} in cell {point[0], point[1]} with a garbage amount: {model.b[point[2], point[3]].value}")


Truck 0 with capacity 9
Step 0: Truck 0 in cell (0, 0) with a garbage amount: 0.0
Step 1: Truck 0 in cell (0, 1) with a garbage amount: 0.0
Step 2: Truck 0 in cell (0, 2) with a garbage amount: 0.0
Step 3: Truck 0 in cell (0, 3) with a garbage amount: 0.0
Step 4: Truck 0 in cell (0, 4) with a garbage amount: 2.0
Step 5: Truck 0 in cell (1, 4) with a garbage amount: 2.0
Step 6: Truck 0 in cell (2, 4) with a garbage amount: 2.0
Step 7: Truck 0 in cell (3, 4) with a garbage amount: 2.0
Step 8: Truck 0 in cell (4, 4) with a garbage amount: 6.0
Step 9: Truck 0 in cell (4, 5) with a garbage amount: 6.0
Step 10: Truck 0 in cell (5, 5) with a garbage amount: 6.0
Step 11: Truck 0 in cell (6, 5) with a garbage amount: 9.0
Step 12: Truck 0 in cell (5, 5) with a garbage amount: 9.0
Step 13: Truck 0 in cell (4, 5) with a garbage amount: 9.0
Step 14: Truck 0 in cell (3, 5) with a garbage amount: 9.0
Step 15: Truck 0 in cell (3, 4) with a garbage amount: 9.0
Step 16: Truck 0 in cell (3, 3) with a ga

In [14]:
import time  

# Colores ANSI
RESET = "\033[0m"
RED = "\033[31m"  # Rojo para el paso actual del camión
CYAN = "\033[36m"  # Cian para celdas previamente visitadas
GREEN = "\033[32m"  # Verde para celdas con basura

# Recorrido del camión paso por paso
for k in model.truck:
    matriz = [[f"{GREEN}{model.dirt_regular_stops[i, j]}{RESET}" if model.dirt_regular_stops[i, j] > 0 else " " 
           for j in range(N_COL)] 
          for i in range(N_ROW)]
    visitadas = [[False for _ in range(N_COL)] for _ in range(N_ROW)]
    print(f"ROUTE OF TRUCK {k+1}")

    for s in range(int(model.return_step[k].value) + 1):
        # Crear una copia para marcar el paso actual sin modificar la matriz original
        matriz_temp = [fila.copy() for fila in matriz]

        # Buscar la celda visitada en este paso
        for i in model.row:
            for j in model.column:
                if model.r[i, j, k, s].value == 1:
                    matriz_temp[i][j] = f"{RED}X{RESET}"  # Marcar la celda actual en rojo
                    visitadas[i][j] = True  # Marcar la celda como visitada

        # Actualizar celdas previamente visitadas a cian
        for i in range(N_ROW):
            for j in range(N_COL):
                if visitadas[i][j] and matriz_temp[i][j] != f"{RED}X{RESET}":  
                    matriz_temp[i][j] = f"{CYAN}X{RESET}"

        # Mostrar la matriz actualizada
        for fila in matriz_temp:
            print("+---" * N_COL + "+")
            print("| " + " | ".join(fila) + " |")
        print("+---" * N_COL + "+\n")
        
        # Esperar antes de continuar al siguiente paso
        time.sleep(0.5) 

ROUTE OF TRUCK 0
+---+---+---+---+---+---+---+---+
| X |   |   |   | 2 |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   | 3 |   |   |   | 2 |   |
+---+---+---+---+---+---+---+---+
|   |   |   | 4 |   | 3 |   |   |
+---+---+---+---+---+---+---+---+
|   | 3 |   |   |   |   |   | 1 |
+---+---+---+---+---+---+---+---+
|   |   | 5 |   | 4 |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   | 4 |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
| 2 |   |   |   |   | 3 |   |   |
+---+---+---+---+---+---+---+---+
|   | 5 |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+

+---+---+---+---+---+---+---+---+
| X | X |   |   | 2 |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   | 3 |   |   |   | 2 |   |
+---+---+---+---+---+---+---+---+
|   |   |   | 4 |   | 3 |   |   |
+---+---+---+---+---+---+---+---+
|   | 3 |   |   |   |   |   | 1 |
+---+---+---+---+---+---+---+---+
|   |   | 5 |   | 4 |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   | 4 |   |   |   |   | 